### Optimal pooling example

Some characteristics of optimal pooling:
- **Different bins**: Bins should be distinct from each other and not overlap. This ensures that the continuous variable is properly discretized and can help avoid confusion when interpreting the results.
- **Balanced distribution**: The number of observations in each bin should be roughly equal or at least not highly skewed. This helps ensure that each bin contributes meaningfully to the model and prevents any single bin from dominating the results.
- **Minimal Gini decrease**: The Gini index is a measure of statistical dispersion, often used to assess the quality of a split in decision trees. In optimal binning, a smaller decrease in the Gini index indicates that the bins are more homogeneous, leading to better model performance.
- **Monotonic relationship**: Ideally, bins should have a monotonic relationship with the target variable, meaning that the relationship should consistently increase or decrease as the bin values change. This simplifies interpretation and can improve model performance.
- **Interpretability**: The bins should be easy to understand and interpret, with meaningful boundaries that are relevant to the domain of the problem. This helps stakeholders better understand the results and can aid in decision-making.
- **Stability**: Good optimal binning should be stable and not sensitive to small changes in the data. This ensures that the model's performance is robust and reliable.
- **Domain knowledge**: The binning process should take into account domain knowledge and expert input when determining the bin boundaries. This can help ensure that the bins are meaningful and relevant to the problem at hand.

-----------------------------

#### Objective Function

Maximize IV value of the bins

IV = ∑((Good% - Bad%) * ln(Good% / Bad%)), where Good% and Bad% are the proportions of non-default and default observations, respectively, in each bin.

**Constraints**:
- The default rate per bin should be monotonically increasing or decreasing.
- The bins should be statistically different (determined by the Chi-square test or another suitable hypothesis test with a chosen significance level).
- The minimum number of observations per bin should be met.
- The minimum and maximum number of bins should be within the specified range.

**Variables**:
- Bins as a set of ordinal categories.
- Default rate: The proportion of default observations (default_flag = 1) in each bin.
- A similar problem is solved here: https://github.com/guillermo-navas-palencia/optbinning/blob/master/optbinning/binning/cp.py

In [1]:
from ortools.sat.python import cp_model
from scipy import special
from sklearn.tree import DecisionTreeRegressor, _tree
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score

import pandas as pd
import numpy as np
import random

pd.set_option('display.max_columns', None)
pd.set_option('max_colwidth', None)

### A. Create a synthetic dataset

In [2]:
# Set a seed so the randomness is reproducible
np.random.seed(0)

# Number of samples
n_samples = 1_000_000

# Generate columns
target = np.random.binomial(1, 0.05, n_samples)
year = np.random.randint(2010, 2023, n_samples)

x1 = np.random.rand(n_samples)
x1_noise = np.random.normal(0, 0.5, n_samples)
x1 = 0.5 * x1 + 0.5 * target + x1_noise

x2 = np.sin(3*np.pi*np.random.rand(n_samples))
x2_noise = np.random.normal(0, 0.5, n_samples)
x2 = 0.5 * x2 + 0.5 * target + x2_noise

x3_noise = np.random.normal(0, 0.5, n_samples)
x3 = 0.5 * x1 + 0.1 * target + x1_noise

# Consolidate all the columns into a DataFrame
df = pd.DataFrame({
    'x1': x1,
    'x2': x2,
    'x3': x3,
    'year': year,
    'target': target,
})

# Define the logistic regression model
model = LogisticRegression()

# Fit the model
X = df[['x1', 'x2']]
y = df['target']
model.fit(X, y)

# Make predictions (probabilities)
y_pred_proba = model.predict_proba(X)[:, 1]
df['score'] = y_pred_proba

# Compute Gini coefficient
gini = 2*roc_auc_score(y, y_pred_proba) - 1
print(f"Gini: {gini:.2%}")

Gini: 63.07%


### B. Create possible cut-off thresholds and annual default rates

This step can be replaced by equal-width / equal-height / xgboost binning

In [3]:
# Set the number of bins you want
n_bins = 100

# Set the max_depth, min_samples_leaf, and max_leaf_nodes parameters
clf = DecisionTreeRegressor(max_depth=n_bins, 
                            min_samples_leaf=1000, 
                            max_leaf_nodes=n_bins)

clf.fit(df[['score']], df['target'])
thresholds = clf.tree_.threshold[clf.tree_.threshold > _tree.TREE_UNDEFINED]

# Add minimum and maximum edges
bins = sorted([-np.inf] + list(thresholds) + [np.inf])
df['bins'] = pd.cut(df['score'], bins, labels=False)

print([round(num, 4) for num in bins])

[-inf, 0.0032, 0.0069, 0.0109, 0.0137, 0.0173, 0.0223, 0.0252, 0.0326, 0.0378, 0.0381, 0.0382, 0.0499, 0.0586, 0.059, 0.0676, 0.0679, 0.0683, 0.0686, 0.0765, 0.0833, 0.0838, 0.092, 0.0925, 0.0933, 0.0941, 0.0946, 0.0954, 0.0966, 0.0986, 0.0998, 0.1017, 0.1043, 0.105, 0.106, 0.1071, 0.1078, 0.1087, 0.1096, 0.1121, 0.1136, 0.1248, 0.1258, 0.1278, 0.1287, 0.1297, 0.1307, 0.1323, 0.1335, 0.1354, 0.1369, 0.1388, 0.1449, 0.1462, 0.148, 0.1503, 0.152, 0.1536, 0.1553, 0.1581, 0.1598, 0.1615, 0.167, 0.1697, 0.1721, 0.177, 0.179, 0.1811, 0.1839, 0.1862, 0.1896, 0.1935, 0.1997, 0.2022, 0.205, 0.2078, 0.211, 0.221, 0.2268, 0.2347, 0.2415, 0.2472, 0.2514, 0.2777, 0.2842, 0.2901, 0.3023, 0.3134, 0.322, 0.3331, 0.3451, 0.3574, 0.3847, 0.4012, 0.4198, 0.4394, 0.4681, 0.5074, 0.5454, 0.6187, inf]


In [4]:
drs = df.pivot_table(index='year', columns='bins', values='target', aggfunc='mean', margins='all').T
drs.style.format("{:.2%}")

year,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022,All
bins,,,,,,,,,,,,,,
0,0.15%,0.19%,0.21%,0.19%,0.35%,0.09%,0.17%,0.30%,0.19%,0.25%,0.24%,0.40%,0.22%,0.23%
1,0.47%,0.44%,0.58%,0.55%,0.53%,0.73%,0.57%,0.49%,0.63%,0.49%,0.42%,0.46%,0.56%,0.53%
2,0.82%,0.98%,1.03%,0.88%,1.03%,0.80%,0.86%,1.07%,0.99%,0.88%,0.73%,0.85%,1.17%,0.93%
3,1.39%,1.68%,1.51%,1.09%,1.07%,1.54%,1.05%,1.41%,1.24%,1.65%,1.09%,1.18%,1.19%,1.31%
4,1.33%,1.81%,1.57%,1.76%,1.63%,1.76%,1.50%,1.53%,1.31%,1.76%,1.74%,1.60%,1.41%,1.59%
5,1.94%,1.88%,1.83%,2.27%,1.72%,1.92%,2.00%,1.99%,2.32%,2.39%,1.90%,1.92%,2.03%,2.01%
6,2.13%,2.57%,2.37%,2.64%,2.49%,2.29%,2.89%,2.67%,2.00%,2.52%,2.00%,2.47%,2.45%,2.42%
7,2.79%,3.12%,2.90%,3.23%,2.55%,2.91%,2.82%,2.60%,2.99%,2.69%,2.83%,2.99%,2.86%,2.87%
8,3.92%,3.50%,2.73%,3.75%,3.82%,4.00%,3.34%,3.67%,3.36%,3.85%,3.07%,3.45%,3.27%,3.52%


### C. Create optimisation function
Some functions are inspired by optbinning.

In [5]:
def calculate_iv(events, non_events, total_events, total_non_events):
    # Calculate the percentage of events and non-events
    event_rate = events / total_events
    non_event_rate = non_events / total_non_events

    # Add a constant to the denominator to avoid division by zero
    event_rate = np.where(event_rate == 0, 0.000001, event_rate)
    
    WoE = np.log(non_event_rate / event_rate)
    IV = (non_event_rate - event_rate) * WoE

    return IV

def test_proportions(event_1, non_event_1, event_2, non_event_2):
    total_1 = event_1 + non_event_1
    total_2 = event_2 + non_event_2
    proportion_1 = event_1 / total_1
    proportion_2 = event_2 / total_2
    pooled_proportion = (event_1 + event_2) / (total_1 + total_2)

    z = ((proportion_1 - proportion_2) / np.sqrt(pooled_proportion * 
         (1 - pooled_proportion) * (1 / total_1 + 1 / total_2)))
    return abs(z)              
                
def individual_bin_stats(events, non_events):
    ''' Create stats for all purposes'''
    n = len(events)
    stats = {}
    
    counts = events + non_events
    total_events = events.sum()
    total_non_events = (counts - events).sum()
    

    for i in range(n+1):
        for j in range(i, n):
            # Calculate the combined events and non-events for each bin combination
            combined_events = events[i:j+1].sum()
            combined_non_events = (counts[i:j+1] - events[i:j+1]).sum()
            
            # print(12, i, j, combined_events, combined_non_events, total_events, total_non_events) # todo: combined nonevent a problem
            # print(78, counts[i:j+1], events[i:j+1], combined_non_events)

            # Calculate IV for this bin combination
            iv = calculate_iv(combined_events, combined_non_events, total_events, total_non_events)
            default_rate = combined_events / (combined_events + combined_non_events)
                
            stats[(i+1, j+1)] = [iv, 
                                 combined_events, 
                                 combined_non_events, 
                                 total_events, 
                                 total_non_events, 
                                 default_rate]

    return stats

def bin_combo_stats(stats_b):
    result_stats = {}
    for a in stats_b:
        for b in stats_b:
            if a[1] == b[0]-1:
                e1 = stats_b[a][1]
                ne1 = stats_b[a][2]
                e2 = stats_b[b][1]
                ne2 = stats_b[b][2]
                
                # Calculate mononicity of consecutive default rates
                default_rate_1 = stats_b[a][5]
                default_rate_2 = stats_b[b][5]
                trend = 0 if default_rate_1 > default_rate_2 else 1
                
                # CaLculate z-score of adjacent bins
                z_score = test_proportions(e1, ne1, e2, ne2)
                
                # Store results
                result_stats[a, b] = [z_score, trend]
                
    return result_stats
                
class VarArraySolutionPrinter(cp_model.CpSolverSolutionCallback):
    """ Intermediate solutions for debugging and multiple solutions """

    def __init__(self, variables):
        cp_model.CpSolverSolutionCallback.__init__(self)
        self.__variables = variables
        self.__solution_count = 0

    def on_solution_callback(self):
        self.__solution_count += 1
        for v in self.__variables:
            pass
            

def create_pools(events, non_events, lra_drs, min_bins, max_bins):
    ''' Finds optimal binning given potential cutoffs '''
    
    n = len(events)
    
    # Create the model.
    model = cp_model.CpModel()

    # Variables
    x = {}
    iv = {}
    bins = []
    for i in range(1, n+1):
        for j in range(i, n+1):
            # x[i, j] is a boolean that is true if included in the combination
            x[i, j] = model.NewBoolVar(f'x[{i},{j}]')
    
    # Create supporting data
    bin_data = individual_bin_stats(events, non_events)        
    iv = {key: val[0] for key, val in bin_data.items()}
    
    # Testing: todo delete
    # iv[2, 4] = 12007
    
    # Create bin options that should be excluded from consideration
    excluded_bin_combos = bin_combo_stats(bin_data)
    
    # Testing: todo delete
    # excluded_bin_combos[(2, 4), (5, 6)] = [0.1, 1]
    # excluded_bin_combos[(2, 4), (5, 5)] = [0.1, 0]
    
    # Constraints
    for i in range(1, n+1):
        # Each bin i should be included in exactly one tuple.
        model.Add(sum(x[j, k] for j in range(1, i+1) for k in range(i, n+1)) == 1)
        
    # Reject bins if the z-score is below the desired threshold (todo: check signs)
    for key, value in excluded_bin_combos.items():
        if value[0] < 1.0:
            ((i1, j1), (i2, j2)) = key
            model.Add(x[i1, j1] + x[i2, j2] <= 1)
            
    # Reject bins if the default rates are not monotonically increasing
    # for key, value in excluded_bin_combos.items():
    #     if value[1] == 1:
    #         ((i1, j1), (i2, j2)) = key
    #         model.Add(x[i1, j1] + x[i2, j2] <= 1)    

    # Maximum number of bins (max_bins)
    model.Add(sum(x[i, j] for i in range(1, n+1) for j in range(i, n+1)) <= max_bins)

    # Minimum number of bins (min_bins)
    model.Add(sum(x[i, j] for i in range(1, n+1) for j in range(i, n+1)) >= min_bins)


    # Objective function
    model.Maximize(sum(iv[i, j] * x[i, j] for i in range(1, n+1) for j in range(i, n+1)))

    # Create a solver and solve.
    solver = cp_model.CpSolver()
    solution_printer = VarArraySolutionPrinter(x)
    status = solver.SolveWithSolutionCallback(model, solution_printer)

    if status == cp_model.OPTIMAL:
        print('Total IV =', solver.ObjectiveValue(), cp_model.OPTIMAL, cp_model.FEASIBLE)
        for i in range(1, n+1):
            for j in range(i, n+1):
                if solver.Value(x[i, j]) == 1:
                    print(i, j)
                    bins.append([i-1, j-1])

    return bins, status, solver, iv, bin_data, excluded_bin_combos

### D. Solve the optimisation task

In [6]:
# Get input data per bin
stats_combined = df.groupby('bins').agg({'target': ['sum', 'count']})
stats_combined = stats_combined.reset_index()
stats_combined.columns = ['bins', 'events', 'count']
stats_combined['non_events'] = stats_combined['count'] - stats_combined['events']
stats_combined['default_rate'] = stats_combined['events'] / stats_combined['count']

# Add long-run averages
annual_averages = df.groupby(['bins', 'year'])['score'].mean()
average_scores_per_bin = annual_averages.groupby('bins').mean().reset_index()
average_scores_per_bin.columns = ['bins', 'lra_dr']

stats_combined = pd.merge(stats_combined, average_scores_per_bin, on='bins', how='left')

stats_combined

,bins,events,count,non_events,default_rate,lra_dr
0,0,138,60833,60695,0.002269,0.002004
1,1,573,107568,106995,0.005327,0.005026
2,2,963,103584,102621,0.009297,0.008849
3,3,802,61059,60257,0.013135,0.012315
4,4,1051,65992,64941,0.015926,0.015475
...,...,...,...,...,...,...
95,95,532,1244,712,0.427653,0.453621
96,96,679,1380,701,0.492029,0.486107
97,97,578,1005,427,0.575124,0.525269
98,98,767,1268,501,0.604890,0.578872


In [7]:
# Test solution
events = stats_combined['events']
non_events = stats_combined['non_events']
lra_drs = stats_combined['lra_dr']

bins, _, _, iv, bin_data, _ = create_pools(events, non_events, lra_drs, 5, 20)

Total IV = 1.611533375983667 4 2
1 1
2 2
3 3
4 4
5 5
6 6
7 8
9 9
10 12
13 17
18 21
22 38
39 52
53 66
67 82
83 89
90 93
94 97
98 99
100 100


In [8]:
# Create new bins
new_bins = [edge for sublist in bins for edge in sublist]
new_bins[-1] += 1  # To include the last edge in the bin

def map_bins(row, bin_edges):
    for i in range(0, len(bin_edges)-1):
        if bin_edges[i] <= row['bins'] < bin_edges[i+1]:
            return i // 2
    return (len(bin_edges) - 2) // 2

df['new_bins'] = df.apply(map_bins, bin_edges=new_bins, axis=1)

# Show stats
df.groupby('new_bins').agg({'bins':['min', 'max']})

bins    
          min max
new_bins         
0           0   0
1           1   1
2           2   2
3           3   3
4           4   4
5           5   5
6           6   7
7           8   8
8           9  11
9          12  16
10         17  20
11         21  37
12         38  51
13         52  65
14         66  81
15         82  88
16         89  92
17         93  96
18         97  98
19         99  99

In [9]:
bins

[[0, 0],
 [1, 1],
 [2, 2],
 [3, 3],
 [4, 4],
 [5, 5],
 [6, 7],
 [8, 8],
 [9, 11],
 [12, 16],
 [17, 20],
 [21, 37],
 [38, 51],
 [52, 65],
 [66, 81],
 [82, 88],
 [89, 92],
 [93, 96],
 [97, 98],
 [99, 99]]

In [10]:
df.groupby('new_bins').agg({'target':['count', 'mean']})

target          
           count      mean
new_bins                  
0          60833  0.002269
1         107568  0.005327
2         103584  0.009297
3          61059  0.013135
4          65992  0.015926
5          75988  0.020095
6         115864  0.027239
7          44587  0.035190
8          78352  0.043024
9          78036  0.056474
10         43570  0.074891
11         48335  0.094404
12         39415  0.122542
13         22862  0.156723
14         26451  0.206911
15         13308  0.291404
16          5783  0.380252
17          4733  0.465033
18          2273  0.591729
19          1407  0.737029